クロスデバイス　Valis拡張との併用

In [2]:
# ライブラリimport
import argparse
import logging
import os

import pandas as pd
from pandas import DataFrame
import pandas.tseries.offsets as offsets
import numpy as np
from datetime import date, timedelta, time, datetime
from tabulate import tabulate

from itertools import product

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

from collections import OrderedDict

import zipfile
import glob

import gc

from ailab_tools.magics import slack_notify

8808 #GROOVE X株式会社

In [4]:
today_yymmdd = date.today().strftime("%y%m%d")
print(today_yymmdd)

190423


In [26]:
#デバイス
seed_sp_device = 'iOS','Android'
xd_sp_device = 'iphone','android_phone','windows_phone','android_tablet','ipad','windows_tablet'

#ターゲット
advertiser_id = 8808 #GROOVE X株式会社
#conversion_group_id = 13054

#grp = 'cvg'
#grp = 'rtg'

In [27]:
print(xd_sp_device)

('iphone', 'android_phone', 'windows_phone', 'android_tablet', 'ipad', 'windows_tablet')


セグメントIDを確認する

In [6]:
#f-string
query = (
    f"""
    select *
    from 
    console.segment
    where 
    org_segment_id in ("30008210")

    """
    )

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df_seg = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [7]:
df_seg.head()

,id,segment_owner_id,dmp_id,org_segment_id,name,status,import_file_name,created_at,updated_at,version
0,9692,1961,20020,30008210,[V拡張][SP]lockon：体験会,1,,2019-03-23 22:06:43,2019-03-23 22:06:43,1


SPセグメントユーザー

In [8]:
segment_id = 9692

In [13]:
seed_table = f"dm_tmp.ya_seed_sp_advid_{advertiser_id}_segid_{segment_id}_{today_yymmdd}"
print(seed_table)

dm_tmp.ya_seed_sp_advid_8808_segid_9692_190423


In [17]:
%%slack_notify @yusuke_akada

#f-string
query = (
    f"""
    select
    smn_uid
    ,val
    from 
    dsplog_sampling.segment_and_ata_uid_sampling
    where 
    concat_ws('-', year, month, day) between '2019-04-20' and '2019-04-22'
    and type in ("segment_ids")
    and key = {segment_id}
    order by val desc
    limit 100000
    """
    )

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df = ir.sql_to_pandas(query)
        ir.upload_dataframe(seed_table, df, ['bigint'])
        
#df_seed = df.drop(['conversion_group_id','device', 'advertiser_id'], axis=1)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [18]:
df.shape[0]

100000

In [19]:
df.head()

,smn_uid,val
0,2186000072838004,10000
1,1147000334964003,10000
2,2162000020315005,10000
3,1149000687188006,10000
4,2186000072838004,10000


In [20]:
df = df.drop("val", axis=1)

In [21]:
df.head()

,smn_uid
0,2186000072838004
1,1147000334964003
2,2162000020315005
3,1149000687188006
4,2186000072838004


In [22]:
df.to_csv(f'{seed_table}.tsv', sep='\t', index=False)

In [23]:
#クロスデバイステーブル
xd_table = 'segment.drawbridge_device'

拡張上位ユーザーをクロスデバイス(SP→PC)

In [25]:
uid_table = f"dm_tmp.ya_xd_pc_advid_{advertiser_id}_segid_{segment_id}_{today_yymmdd}"
print(uid_table)

dm_tmp.ya_xd_pc_advid_8808_segid_9692_190423


In [28]:
%%slack_notify @yusuke_akada
#XD拡張

#f-string
query = (
    f"""
    select 
    distinct xd2.smn_uid
    from {seed_table} sd
    left join {xd_table} xd
    on sd.smn_uid = xd.smn_uid
    left join {xd_table} xd2
    on xd.drawbridge_consumer_id = xd2.drawbridge_consumer_id
    where xd2.device not in {xd_sp_device}
    limit 10000
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_fix = ir.sql_to_pandas(query)
    #ImpalaにUp
    #ir.upload_dataframe(uid_table, df_fix, ['bigint'])
    #テーブル削除
    ir.drop_tables([seed_table])
    
df_fix.to_csv(f'{uid_table}.tsv', sep='\t', index=False, header=False)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [29]:
df_fix.shape[0]

2340

In [ ]:
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    ir.drop_tables([uid_table])